## Tarea 2 (10 puntos). Operaciones con RDDs
- Obtener información de algún origen de datos (propio o de API)
- Convertir el origen de datos a RDD con pySpark
- Realizar alguna operación en el RDD, como estadísticas descriptivas básicas
- Subir la práctica a un repositorio público y etiquetarla claramente

In [1]:
import os
import sys

import findspark
findspark.init()
findspark.find()

from pyspark import SparkContext

from pyspark.sql import SparkSession

In [2]:
import multiprocessing

multiprocessing.cpu_count()

16

In [3]:
from pyspark import SparkConf

In [4]:
conf = SparkConf().setAppName("Tarea2")\
    .set("spark.master","local[10]")\
    .set("spark.executor.cores","2")\
    .set("spark.executor.memory", "4g")

In [5]:
#sc = SparkContext("local", "RDDs")
sc = SparkContext(conf= conf)

In [6]:
spark = SparkSession(sparkContext= sc)

In [8]:
columnas = [
    'Airline', # Aerolinea
    'CRSDepTime', #Hora de salida programada
    'DepTime', #Hora de salida actual
    'DepDelay', #Diferencia en minutos entre CRSDepTime y DepTime, tiempos negativos significa salidas tempranas
    'CRSArrTime',
    'ArrTime', #Hora de llegada
    'ArrDelay', #Diferencia de minutos entre la llegada programada y la hora de llegada, tiempos negativos significa llegadas tempranas
    'ActualElapsedTime', #Tiempo de vuelo verdadero
    'CRSElapsedTime', #Tiempo de vuelo programado
    'Distance', #Distancia entre aeropuertos en millas
    'Year', #Año del vuelo
    'Month', #Mes del vuelo
    'DayofMonth', #Dia del mes del vuelo
    'DayOfWeek', #Dia de la semana del vuelo
    'Tail_Number', #Numero de la cola, código para identificar la aeronave
    'Flight_Number_Operating_Airline', #Numero del vuelo
    
    'Origin', #Aeropuerto de origen
    'OriginAirportID', 'OriginAirportSeqID', #codigos del aeropuerto de origen
    'OriginCityName', #Nombre de la ciudad del aeropuerto de origen
    'OriginStateName', #Nombre de estado del aeropuerto de origen
    
    'Dest', #Aeropuerto de destino
    'DestAirportID', 'DestAirportSeqID', #codigos del aeropuerto de destino
    'DestCityName', #Nombre de la ciudad del aeropuerto de destino
    'DestStateName', #Nombre de estado del aeropuerto de destino
    
    'Cancelled', #El vuelo fue cancelado, 1 = Sí
    
    'Diverted', #Si el vuelo fue desviado, 1 = Sí
]

In [11]:
import zipfile
import pathlib

direccion = pathlib.Path.cwd()

path_datos_zip = pathlib.Path.joinpath(direccion.parent.parent, 'Datos\\data.zip')

path_guardar = pathlib.Path.joinpath(direccion.parent.parent, 'Datos')

archivos_datos = [
    pathlib.Path.joinpath(direccion.parent.parent, 'Datos\\Combined_Flights_2022.csv'),
    pathlib.Path.joinpath(direccion.parent.parent, 'Datos\\Combined_Flights_2021.csv'),
    pathlib.Path.joinpath(direccion.parent.parent, 'Datos\\Combined_Flights_2020.csv'),
    pathlib.Path.joinpath(direccion.parent.parent, 'Datos\\Combined_Flights_2019.csv'),
    pathlib.Path.joinpath(direccion.parent.parent, 'Datos\\Combined_Flights_2018.csv')
]

archivos_datos_en_zip  = [
    'Combined_Flights_2022.csv',
    'Combined_Flights_2021.csv',
    'Combined_Flights_2020.csv',
    'Combined_Flights_2019.csv',
    'Combined_Flights_2018.csv'
]


In [ ]:
with zipfile.ZipFile(path_datos_zip, mode="r") as archivo_zip:
    for archivo in archivos_datos_en_zip:
        archivo_zip.extract(archivo, path_guardar)

In [23]:
def leer_archivo(archivo):
    direccion = pathlib.Path.cwd()
    direccion_archivo =  pathlib.Path.joinpath(direccion.parent.parent, 'Datos',archivo)
    
    return spark.read.csv(direccion_archivo)

In [24]:
rdd = spark.sparkContext.parallelize(archivos_datos)
rdd_contenido = rdd.map(lambda archivo: {"archivo": archivo,"datos": leer_archivo(archivo)})


In [20]:
rdd = spark.sparkContext.parallelize(archivos_datos)
rdd_contenido = rdd.map(lambda archivo: leer_archivo(archivo))

In [25]:
rdd_contenido.count()

Traceback (most recent call last):
  File "c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyspark\serializers.py", line 459, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyspark\cloudpickle\cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyspark\cloudpickle\cloudpickle_fast.py", line 632, in dump
    return Pickler.dump(self, obj)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyspark\context.py", line 466, in __getnewargs__
    raise PySparkRuntimeError(
pyspark.errors.exceptions.base.PySparkRuntimeError: [CONTEXT_ONLY_VALID_ON_DRIVER] It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkConte

PicklingError: Could not serialize object: PySparkRuntimeError: [CONTEXT_ONLY_VALID_ON_DRIVER] It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on workers. For more information, see SPARK-5063.